In [50]:
import csv


from pathlib import Path

directory = Path("HomographData/train/")

# Initialize lists to store input_texts and target_texts
dataTexts = []


for file in directory.iterdir():
    if file.is_file():  # Ensure it's a file, not a subdirectory
        # print(file)
        filepath = Path(file)
        name_without_ext = filepath.stem  # Extract file name without extension
        # print(name_without_ext) 

        # Open and read the TSV file
        with open(file, "r", encoding="utf-8") as currentfile:
            reader = csv.DictReader(currentfile, delimiter="\t")
            for row in reader:
                # Construct input_text and target_text
                exampleText = {}

                startpos = int(row["start"])
                endpos = int(row["end"])
                exampleText['sentence'] = row["sentence"]
                exampleText['word'] = name_without_ext
                exampleText['target'] = row["wordid"]

                # print(exampleText)
                # Add to respective lists
                dataTexts.append(exampleText)

# Example output
print(" examples:", dataTexts[:3])  # Display first 2 inputs


 examples: [{'sentence': 'Smith uses his name as a base for building abstract imagery.', 'word': 'abstract', 'target': 'abstract_adj-nou'}, {'sentence': 'The Group intended to promote Welsh radical and abstract modern art (and successfully did so into the 21st century).', 'word': 'abstract', 'target': 'abstract_adj-nou'}, {'sentence': 'Both stances are verbalized with the same abstract vocabulary.', 'word': 'abstract', 'target': 'abstract_adj-nou'}]


In [56]:
# from flair.models import SequenceTagger
from flair.data import Sentence
from flair.models import SequenceTagger
from flair.nn import Classifier

import ast

# Load the POS tagger
tagger = Classifier.load("pos")
# tagger = SequenceTagger.load("flair/ner-multi")

# dataWinds = [
#     {
#         "predicted_wordid": "winds_nou",
#         "target": "winds_vrb",
#         "sentence": "He sees Isabelle mingling, but bumps straight into Kay, an ex-girlfriend, who winds him up, lambasting him."
#     },
#     {
#         "predicted_wordid": "winds_nou",
#         "target": "winds_vrb",
#         "sentence": "The continuation of the street winds up to the tomb of Samuel the prophet, after whom the street is named."
#     },
#     {
#         "predicted_wordid": "winds_nou",
#         "target": "winds_vrb",
#         "sentence": "Besides the big, wild, and connected lands, ACT proposed the Hogtown Creek Greenway, which winds through western Gainesville."
#     },
#     {
#         "predicted_wordid": "winds_nou",
#         "target": "winds_vrb",
#         "sentence": "The road winds through the scenic, mountainous country of New Hampshire north of the White Mountain National Forest."
#     }
# ]

def extract_list_after_arrow(text):
    # Split the text using the arrow as a delimiter
    parts = text.split("→")
    
    # Check if the arrow exists in the text
    if len(parts) > 1:
        # Extract the part after the arrow and strip any leading/trailing whitespace
        list_str = parts[1].strip()
# Preprocess to convert it into valid Python list syntax
        processed_str = list_str.replace("/", "").replace(",,", ",")
        
        # Manually parse the string to generate the list
        items = processed_str[1:-1].split(", ")
        parsed_list = [item.strip('"') for item in items]
        
        return parsed_list    
    else:
        # Return a message if the arrow isn't found
        return "Arrow not found in the text."

# # Example input
# text = 'He sees Isabelle mingling, but bumps straight into Kay, an ex-girlfriend, who winds him up, lambasting him. → ["He"/PRP, "sees"/VBZ, "Isabelle"/NNP, "mingling"/VBG, ","/,, "but"/CC, "bumps"/VBZ, "straight"/RB, "into"/IN, "Kay"/NNP, ","/,, "an"/DT, "ex-girlfriend"/NN, ","/,, "who"/WP, "winds"/VBZ, "him"/PRP, "up"/RP, ","/,, "lambasting"/VBG, "him"/PRP, "."/.]'

# # Call the function and print the result
# taggedList = extract_list_after_arrow(text)
# print(taggedList)
# print(type(taggedList))

nRight = 0
nWrong = 0

for example in dataTexts:
    # Create a sentence
    sentence = Sentence(example['sentence'])

    # Predict POS tagss
    tagger.predict(sentence)

    foundWord = False

    for token in sentence:
        # print('word' , example['word'] , 'token text' , token.text.lower())
       word = example['word']

       if token.text.lower() == word:
            foundWord = True
            # print(f"Word: {token.text}, POS Tag: {token.labels}")
            # print(type(token.labels[0]) , token.labels[0])
            # print('value' , token.labels[0].value)
            # print('labid' , token.labels[0].labeled_identifier)
            # print('score' , token.labels[0].score)
            if token.labels[0].value in ['NN' , 'NNS', 'NNP' , 'NNPS']  and example['target'] in [word + '_nou' , word + '_adj-nou']:
                nRight += 1
            elif token.labels[0].value in ['VB' , 'VBP' , 'VBZ']  and example['target'] == word + '_vrb':
                nRight += 1
            elif token.labels[0].value == 'JJ' and example['target'] == word + '_adj-nou':
                nRight += 1
            else:
                print('word POS' , token.text , 'target' , example['target'] , sentence)


    if not foundWord:
        print(word , 'not found in' , sentence)
        # print(f"Word: {token.text}, POS Tag: {token.labels}")
        # print(type(token.labels[0]) , token.labels[0])
        # print('value' , token.labels[0].value)
        # print('labid' , token.labels[0].labeled_identifier)
        # print('score' , token.labels[0].score)

    # break

    # # Print the result
    # taggedSentence = sentence.to_tagged_string()
    # # print(taggedSentence)
    # # print(type(taggedSentence))

    # taggedList = extract_list_after_arrow(taggedSentence)
    # # print(taggedList)
    # # print(type(taggedList))

    # for taggedWord in taggedList:
    #     if 'winds' in taggedWord:
    #         if taggedWord == 'winds"NNS' and windsExample['target'] == 'winds_nou':
    #             nRight += 1
    #         elif taggedWord == 'winds"VBZ' and windsExample['target'] == 'winds_vrb':
    #             nRight += 1
    #         else:
    #             print('winds POS' , taggedWord , 'target' , windsExample['target'] , sentence)

print('nRight' , nRight)


2025-04-01 14:01:58,616 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD
abuse not found in Sentence[10]: "Advocacy services that encourage victims to establish abuse-free lives." → ["Advocacy"/NN, "services"/NNS, "that"/WDT, "encourage"/VBP, "victims"/NNS, "to"/TO, "establish"/VB, "abuse-free"/JJ, "lives"/NNS, "."/.]
addict not found in Sentence[11]: "Bhavani is not interested, and he becomes a drug-addict." → ["Bhavani"/NNP, "is"/VBZ, "not"/RB, "interested"/JJ, ","/,, "and"/CC, "he"/PRP, "becomes"/VBZ, "a"/DT, "drug-addict"/NN, "."/.]
addict not found in Sentence[16]: "She followed Donny, her gambling-addict boyfriend, to Vietnam with a show tour team." → ["She"/PRP, "followed"/VBD, "Donny"/NNP, ","/,, "her"/PRP$, "gambling-addict"/JJ, "boyfriend"/NN, ","/,,